In [1]:
import pandas as pd
import torch

In [2]:
unique_biokg_prot_ids = pd.read_csv('../data/biokg/unique_proteins.csv', 
                                    index_col=0)

In [3]:
unique_biokg_prot_ids

,protein_id
0,P9WHP6
1,Q54KD4
2,Q7Z6G3
3,Q9Y7S6
4,Q9VL31
...,...
122897,P24271
122898,D5A7J3
122899,Q53NK6
122900,P70459


In [4]:
biokg_uniprot_set = set(unique_biokg_prot_ids['protein_id'])

In [5]:
unique_biokg_uniprot_ids =  set(unique_biokg_prot_ids)

In [6]:
swissprot_to_uniprot_ids = pd.read_csv('../data/uniprot.tsv',
                                    sep='\t')

In [7]:
uniprot_emb_set = set(swissprot_to_uniprot_ids['Entry'])

In [ ]:
uniprot_emb_set

In [9]:
len(biokg_uniprot_set.difference(uniprot_emb_set))

82379

In [10]:
len(uniprot_emb_set.difference(biokg_uniprot_set))

444

In [11]:
biokg_proteins_without_emb = biokg_uniprot_set.difference(uniprot_emb_set)

In [12]:
test = pd.DataFrame(biokg_proteins_without_emb)

In [ ]:
# To store the ones we're missing
# test.to_csv('prot_without_emb.csv', index=False, header=False)

In [13]:
# Load all missing proteins
biokg_missing_proteins = pd.read_csv('../data/uniprot_sequences.tsv',
                                    sep='\t')

In [14]:
protein_sequences = biokg_missing_proteins[['From', 'Sequence']]

In [17]:
protein_sequences.head(2)

,From,Sequence
0,Q8X730,MALWGGRFTQAADQRFKQFNDSLRFDYRLAEQDIVGSVAWSKALVT...
1,Q29RM2,MITLITEQLQKQTLDELKCTRFSISLPLPDHADISNCGNPFQLVSE...


In [18]:
import numpy as np
import bio_embeddings

from bio_embeddings.embed import SeqVecEmbedder, ProtTransBertBFDEmbedder, prottrans_t5_embedder, esm_embedder

In [19]:
# IF THIS CELLS EXECUTES FOR THE FIRST TIME, EXPECT A DELAY
prot_trans_embedder = ProtTransBertBFDEmbedder()

In [20]:
# Get the aggregated protein representation

def get_protein_repr(amino_repr):
    # Here we need to go from a collection of amino-acid embeddings to a full protein embedding
    #
    # Example: 
    #    
    #   M : (1,1024)
    #   A : (1,1024)
    #   S : (1,1024)
    #   
    #  Output: An aggregated representation for proteins
    #
    #  Type: Dict(protein_id: (embedding))
    #
    #  e.g Dict(: (LENG8_MOUSE, 1024))
     
    emb_matrix = torch.Tensor(amino_repr)
    protein_emb = torch.mean(emb_matrix, dim=0)

    return protein_emb

In [24]:
protein_sequences_sample = protein_sequences.sample(n=2)

In [25]:
%%time
protein_sequences_sample['embedding'] = protein_sequences_sample['Sequence'].apply(lambda x: prot_trans_embedder.embed(x))

CPU times: user 21.7 s, sys: 460 ms, total: 22.1 s
Wall time: 2.78 s


In [26]:
protein_sequences_sample['Aggregated'] = protein_sequences_sample['embedding'].apply(lambda x: get_protein_repr(x))

In the above example we see a protein of 406 amino-acids is represented by a (406,1024) matrix.

To get the final representation we will make it a (1,1024) by "squashing" the amino-acids together.

In [27]:
protein_sequences_sample.to_csv('../data/processed/protein_sequences.tsv',
                                sep='\t')

In [ ]:
### Now we have to do it for all.